In [ ]:
%reset -f

Chapter 27
Reshaping By Pivoting and Grouping

.pivot_table, .groupby and the pd.crosstab  functions

In [1]:
import catboost as cb
import numpy as np

import pandas as pd

import collections

e:\Users\wgrom\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
def get_uniq_cols(jb):
    counter = collections.defaultdict(list)
    for col in sorted(jb.columns):
        period_count = col.count('.')
        if period_count >= 2:
            part_end = 2
        else:
            part_end = 1
        parts = col.split('.')[:part_end]
        counter['.'.join(parts)].append(col)
    uniq_cols = []
    for cols in counter.values():
        if len(cols) == 1:
            uniq_cols.extend(cols)
    return uniq_cols

In [3]:
#books code
def prep_for_ml(df):
    # remove pandas types
    return (df
     .assign(**{col:df[col].astype(float) 
               for col in df.select_dtypes('number')},
             **{col:df[col].astype(str).fillna('') 
               for col in df.select_dtypes(['object', 'category'])})
    )

In [4]:
def predict_col(df, col):
    df = prep_for_ml(df)
    missing = df.query(f'~{col}.isna()')
    cat_idx = []
    for i,typ in enumerate(df.drop(columns=[col]).dtypes):
        if str(typ) == 'object':
            cat_idx.append(i)
    X = (missing
         .drop(columns=[col])
         .values
        )
    y = missing[col]
    model = cb.CatBoostRegressor(iterations=20, cat_features=cat_idx)
    model.fit(X, y, cat_features=cat_idx)
    pred = model.predict(df.drop(columns=[col]))
    return df[col].where(~df[col].isna(), pred)



In [5]:
# books code
def tweak_jb(jb):
     uniq_cols = get_uniq_cols(jb)
     return (jb
         [uniq_cols]
         .rename(columns=lambda c: c.replace('.', '_'))
         .assign(age=lambda df_:df_.age.str.slice(0,2).astype(float)
                    .astype('Int64'),
                are_you_datascientist=lambda df_:df_
                    .are_you_datascientist
                    .replace({'Yes': True, 'No': False, np.nan: False}),
                company_size=lambda df_:df_.company_size.replace({
                    'Just me': 1, 'Not sure': np.nan,
                    'More than 5,000': 5000, '2–10': 2, '11–50':11,
                    '51–500': 51, '501–1,000':501,
                    '1,001–5,000':1001}).astype('Int64'),
                country_live=lambda df_:df_.country_live
                    .astype('category'),
                employment_status=lambda df_:df_.employment_status
                     .fillna('Other').astype('category'),
                is_python_main=lambda df_:df_.is_python_main
                     .astype('category'),
                team_size=lambda df_:df_.team_size
                    .str.split(r'-', n=1, expand=True)
                    .iloc[:,0].replace('More than 40 people', 41)
                    .where(df_.company_size!=1, 1).astype(float),
                years_of_coding=lambda df_:df_.years_of_coding
                    .replace('Less than 1 year', .5)
                    .str.extract(r'(\d+)').astype(float),
                python_years=lambda df_:df_.python_years
                    .replace('Less than 1 year', .5)
                    .str.extract(r'(\d+)').astype(float),
                python3_ver=lambda df_:df_.python3_version_most
                     .str.replace('_', '.').str.extract(r'(\d\.\d)')
                     .astype(float),
                use_python_most=lambda df_:df_.use_python_most
                     .fillna('Unknown')
               )
        .assign(team_size=lambda df_:predict_col(df_, 'team_size')
             .astype(int))
        .drop(columns=['python2_version_most'])
        .dropna()
        .astype({'age':'int64','company_size':'int64'})   # would throw a 'Int64' error if not converted to 'int64'
    )    
url = 'https://github.com/mattharrison/datasets/raw/master/data/'\
   '2020-jetbrains-python-survey.csv'
jb = pd.read_csv(url)
jb2 = tweak_jb(jb)


C:\Users\wgrom\AppData\Local\Temp\ipykernel_6640\2100465703.py:47: DtypeWarning: Columns (152) have mixed types. Specify dtype option on import or set low_memory=False.
  jb = pd.read_csv(url)


Learning rate set to 0.5
0:	learn: 2.9695218	total: 179ms	remaining: 3.39s
1:	learn: 2.8766539	total: 217ms	remaining: 1.95s
2:	learn: 2.8387189	total: 259ms	remaining: 1.47s
3:	learn: 2.8028751	total: 292ms	remaining: 1.17s
4:	learn: 2.7899957	total: 326ms	remaining: 979ms
5:	learn: 2.7749439	total: 362ms	remaining: 844ms
6:	learn: 2.7719128	total: 398ms	remaining: 739ms
7:	learn: 2.7649792	total: 434ms	remaining: 651ms
8:	learn: 2.7649588	total: 467ms	remaining: 571ms
9:	learn: 2.7630617	total: 500ms	remaining: 500ms
10:	learn: 2.7625779	total: 533ms	remaining: 436ms
11:	learn: 2.7515902	total: 567ms	remaining: 378ms
12:	learn: 2.7513459	total: 603ms	remaining: 325ms
13:	learn: 2.7445634	total: 652ms	remaining: 280ms
14:	learn: 2.7443257	total: 715ms	remaining: 238ms
15:	learn: 2.7423142	total: 763ms	remaining: 191ms
16:	learn: 2.7419143	total: 818ms	remaining: 144ms
17:	learn: 2.7399387	total: 880ms	remaining: 97.7ms
18:	learn: 2.7384296	total: 931ms	remaining: 49ms
19:	learn: 2.738

In [6]:
jb2

,age,are_you_datascientist,company_size,country_live,employment_status,first_learn_about_main_ide,how_often_use_main_ide,ide_main,is_python_main,job_team,main_purposes,missing_features_main_ide,nps_main_ide,python_years,python3_version_most,several_projects,team_size,use_python_most,years_of_coding,python3_ver
1,21,True,5000,India,Fully employed by a company / organization,School / University,Daily,VS Code,Yes,Work in a team,Both for work and personal,"No, it has all the features I need",8.0,3.0,Python 3_6,"Yes, I work on one main and several side projects",2,Software prototyping,3.0,3.6
2,30,False,5000,United States,Fully employed by a company / organization,Friend / Colleague,Daily,Vim,Yes,Work on your own project(s) independently,Both for work and personal,"No, it has all the features I need",10.0,3.0,Python 3_6,"Yes, I work on one main and several side projects",4,DevOps / System administration / Writing autom...,3.0,3.6
10,21,False,51,Other country,Fully employed by a company / organization,School / University,Daily,IntelliJ IDEA,Yes,Work in a team,Both for work and personal,"No, it has all the features I need",10.0,1.0,Python 3_8,"Yes, I work on one main and several side projects",2,Web development,1.0,3.8
11,21,True,51,United States,Fully employed by a company / organization,Online learning platform / Online course,Daily,PyCharm Community Edition,Yes,Work in a team,Both for work and personal,"No, it has all the features I need",9.0,3.0,Python 3_9,"Yes, I work on many different projects",2,Data analysis,3.0,3.9
13,30,True,5000,Belgium,Fully employed by a company / organization,Social network,Daily,VS Code,Yes,Work in a team,Both for work and personal,"No, it has all the features I need",10.0,6.0,Python 3_7,"Yes, I work on many different projects",2,Data analysis,3.0,3.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54456,30,False,1001,Turkey,Fully employed by a company / organization,Friend / Colleague,Daily,PyCharm Community Edition,Yes,Work on your own project(s) independently,Both for work and personal,"No, it has all the features I need",9.0,1.0,Python 3_6,"Yes, I work on many different projects",5,Machine learning,6.0,3.6
54457,21,False,2,Russian Federation,Fully employed by a company / organization,School / University,Daily,Vim,Yes,Work on your own project(s) independently,Both for work and personal,"No, it has all the features I need",10.0,6.0,Python 3_6,"Yes, I work on many different projects",2,Data analysis,1.0,3.6
54459,21,False,1,Russian Federation,Self-employed (a person earning income directl...,Friend / Colleague,Daily,PyCharm Professional Edition,Yes,Work in a team,Both for work and personal,"No, it has all the features I need",10.0,3.0,Python 3_7,"Yes, I work on many different projects",1,Web development,6.0,3.7
54460,30,True,51,Spain,Fully employed by a company / organization,Search engines,Daily,Other,Yes,Work on your own project(s) independently,Both for work and personal,Yes – Please list:,3.0,6.0,Python 3_7,"Yes, I work on many different projects",4,Data analysis,3.0,3.7


In [7]:
jb2.dtypes

age                              int64
are_you_datascientist           object
company_size                     int64
country_live                  category
employment_status             category
first_learn_about_main_ide      object
how_often_use_main_ide          object
ide_main                        object
is_python_main                category
job_team                        object
main_purposes                   object
missing_features_main_ide       object
nps_main_ide                   float64
python_years                   float64
python3_version_most            object
several_projects                object
team_size                        int32
use_python_most                 object
years_of_coding                float64
python3_ver                    float64
dtype: object

In [34]:
(jb2
 .pivot_table(index='country_live',columns='employment_status',
              values='age',aggfunc='mean'))

employment_status,Fully employed by a company / organization,Partially employed by a company / organization,"Self-employed (a person earning income directly from one's own business, trade, or profession)",Working student
country_live,,,,
Algeria,31.200000,24.000000,27.000000,NaN
Argentina,30.632184,30.333333,28.300000,23.000000
Armenia,22.071429,NaN,24.000000,NaN
Australia,32.935622,28.000000,42.360000,24.125000
Austria,31.619565,30.357143,35.000000,25.500000
...,...,...,...,...
United States,32.429163,27.500000,39.324324,21.842697
Uruguay,27.000000,30.250000,30.000000,NaN
Uzbekistan,21.000000,21.000000,NaN,NaN


In [35]:
pd.crosstab(index=jb2.country_live, columns=jb2.employment_status, 
            values=jb2.age, aggfunc='mean')

employment_status,Fully employed by a company / organization,Partially employed by a company / organization,"Self-employed (a person earning income directly from one's own business, trade, or profession)",Working student
country_live,,,,
Algeria,31.200000,24.000000,27.000000,NaN
Argentina,30.632184,30.333333,28.300000,23.000000
Armenia,22.071429,NaN,24.000000,NaN
Australia,32.935622,28.000000,42.360000,24.125000
Austria,31.619565,30.357143,35.000000,25.500000
...,...,...,...,...
United States,32.429163,27.500000,39.324324,21.842697
Uruguay,27.000000,30.250000,30.000000,NaN
Uzbekistan,21.000000,21.000000,NaN,NaN


In [37]:
(jb2
 .groupby(['country_live', 'employment_status'])
 .age 
 .mean()
 .unstack()
)

employment_status,Freelancer (a person pursuing a profession without a long-term commitment to any one employer),Fully employed by a company / organization,Other,Partially employed by a company / organization,Retired,"Self-employed (a person earning income directly from one's own business, trade, or profession)",Student,Working student
country_live,,,,,,,,
Algeria,NaN,31.200000,NaN,24.000000,NaN,27.000000,NaN,NaN
Argentina,NaN,30.632184,NaN,30.333333,NaN,28.300000,NaN,23.000000
Armenia,NaN,22.071429,NaN,NaN,NaN,24.000000,NaN,NaN
Australia,NaN,32.935622,NaN,28.000000,NaN,42.360000,NaN,24.125000
Austria,NaN,31.619565,NaN,30.357143,NaN,35.000000,NaN,25.500000
...,...,...,...,...,...,...,...,...
United States,NaN,32.429163,NaN,27.500000,NaN,39.324324,NaN,21.842697
Uruguay,NaN,27.000000,NaN,30.250000,NaN,30.000000,NaN,NaN
Uzbekistan,NaN,21.000000,NaN,21.000000,NaN,NaN,NaN,NaN


27.2 Using a Custom Aggregation Function

In [15]:
def per_emacs(ser):
    return ser.str.contains('Emacs').sum()/ len(ser) * 100

# when you want to calculate percentage in pandas , can use mean

def per_emacs2(ser):
    return ser.str.contains('Emacs').mean() * 100


In [16]:
(jb2
 .pivot_table(index='country_live', values='ide_main', aggfunc=per_emacs)
)

,ide_main
country_live,
Algeria,0.000000
Argentina,3.669725
Armenia,0.000000
Australia,3.649635
Austria,1.562500
...,...
United States,4.486466
Uruguay,0.000000
Uzbekistan,0.000000


27.3 Multiple Aggregations

In [18]:
(jb2
 .pivot_table(index='country_live', values='age',
              aggfunc=(min, max))
)

,max,min
country_live,,
Algeria,60,18
Argentina,60,18
Armenia,30,18
Australia,60,18
Austria,50,18
...,...,...
United States,60,18
Uruguay,40,21
Uzbekistan,21,21


In [20]:
(jb2
 .groupby('country_live')
 .age 
 .agg([min, max])
)

,min,max
country_live,,
Algeria,18,60
Argentina,18,60
Armenia,18,30
Australia,18,60
Austria,18,50
...,...,...
United States,18,60
Uruguay,21,40
Uzbekistan,21,21


In [22]:
# this method is not recommend but shown

pd.crosstab(jb2.country_live, values=jb2.age, aggfunc=(min, max),
            columns=jb2.assign(val='age').val)

,max,min
val,age,age
country_live,,
Algeria,60,18
Argentina,60,18
Armenia,30,18
Australia,60,18
Austria,50,18
...,...,...
United States,60,18
Uruguay,40,21


In [25]:
#27.4 Per Column Aggregations

(jb2
 .pivot_table(index='country_live', 
              aggfunc=(min, max))
)


C:\Users\wgrom\AppData\Local\Temp\ipykernel_6640\4241490531.py:3: FutureWarning: ['are_you_datascientist', 'employment_status', 'is_python_main'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  (jb2


age     company_size           first_learn_about_main_ide  \
              max min          max min                              max   
country_live                                                              
Algeria        60  18         5000   1  Technical review / Forum / Blog   
Argentina      60  18         5000   1  Technical review / Forum / Blog   
Armenia        30  18         5000   1  Technical review / Forum / Blog   
Australia      60  18         5000   1  Technical review / Forum / Blog   
Austria        50  18         5000   1  Technical review / Forum / Blog   
...            ..  ..          ...  ..                              ...   
United States  60  18         5000   1  Technical review / Forum / Blog   
Uruguay        40  21         5000   2  Technical review / Forum / Blog   
Uzbekistan     21  21         5000   1  Technical review / Forum / Blog   
Venezuela      50  18         5000   1  Technical review / Forum / Blog   
Viet Nam       60  18         5000   1  Technical review / Forum / Blog   

                                                          \
                                                     min   
country_live                                               
Algeria         Online learning platform / Online course   
Argentina       Online learning platform / Online course   
Armenia         Online learning platform / Online course   
Australia       Online learning platform / Online course   
Austria         Online learning platform / Online course   
...                                                  ...   
United States   Online learning platform / Online course   
Uruguay         Online learning platform / Online course   
Uzbekistan                                   Advertising   
Venezuela       Online learning platform / Online course   
Viet Nam        Online learning platform / Online course   

              how_often_use_main_ide                             ide_main  \
                                 max    min                           max   
country_live                                                                
Algeria                       Weekly  Daily                       VS Code   
Argentina                     Weekly  Daily                           Vim   
Armenia                       Weekly  Daily                           Vim   
Australia                     Weekly  Daily                           Vim   
Austria                       Weekly  Daily                           Vim   
...                              ...    ...                           ...   
United States                 Weekly  Daily                           Vim   
Uruguay                        Daily  Daily                           Vim   
Uzbekistan                     Daily  Daily  PyCharm Professional Edition   
Venezuela                    Monthly  Daily                           Vim   
Viet Nam                      Weekly  Daily                           Vim   

                                ... python_years       \
                           min  ...          max  min   
country_live                    ...                     
Algeria                   Atom  ...         11.0  1.0   
Argentina                 Atom  ...         11.0  1.0   
Armenia                   Atom  ...          6.0  1.0   
Australia                 Atom  ...         11.0  1.0   
Austria                   Atom  ...         11.0  1.0   
...                        ...  ...          ...  ...   
United States             Atom  ...         11.0  1.0   
Uruguay                  Other  ...         11.0  1.0   
Uzbekistan       IntelliJ IDEA  ...          3.0  1.0   
Venezuela      Eclipse + Pydev  ...         11.0  1.0   
Viet Nam                  Atom  ...          6.0  1.0   

                                                several_projects  \
                                                             max   
country_live                                                       
Algeria        Yes, I work on one main a

In [28]:
#remove the columns ['are_you_datascientist', 'employment_status', 'is_python_main']  because of the warning
#FutureWarning: ['are_you_datascientist', 'employment_status', 'is_python_main'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
#  (jb2
(jb2.drop(labels=['are_you_datascientist', 'employment_status', 'is_python_main'], axis=1 )
 .groupby('country_live')
 .agg([min, max])
)

age     company_size        \
              min max          min   max   
country_live                               
Algeria        18  60            1  5000   
Argentina      18  60            1  5000   
Armenia        18  30            1  5000   
Australia      18  60            1  5000   
Austria        18  50            1  5000   
...            ..  ..          ...   ...   
United States  18  60            1  5000   
Uruguay        21  40            2  5000   
Uzbekistan     21  21            1  5000   
Venezuela      18  50            1  5000   
Viet Nam       18  60            1  5000   

                              first_learn_about_main_ide  \
                                                     min   
country_live                                               
Algeria         Online learning platform / Online course   
Argentina       Online learning platform / Online course   
Armenia         Online learning platform / Online course   
Australia       Online learning platform / Online course   
Austria         Online learning platform / Online course   
...                                                  ...   
United States   Online learning platform / Online course   
Uruguay         Online learning platform / Online course   
Uzbekistan                                   Advertising   
Venezuela       Online learning platform / Online course   
Viet Nam        Online learning platform / Online course   

                                               how_often_use_main_ide  \
                                           max                    min   
country_live                                                            
Algeria        Technical review / Forum / Blog                  Daily   
Argentina      Technical review / Forum / Blog                  Daily   
Armenia        Technical review / Forum / Blog                  Daily   
Australia      Technical review / Forum / Blog                  Daily   
Austria        Technical review / Forum / Blog                  Daily   
...                                        ...                    ...   
United States  Technical review / Forum / Blog                  Daily   
Uruguay        Technical review / Forum / Blog                  Daily   
Uzbekistan     Technical review / Forum / Blog                  Daily   
Venezuela      Technical review / Forum / Blog                  Daily   
Viet Nam       Technical review / Forum / Blog                  Daily   

                               ide_main                                ...  \
                   max              min                           max  ...   
country_live                                                           ...   
Algeria         Weekly             Atom                       VS Code  ...   
Argentina       Weekly             Atom                           Vim  ...   
Armenia         Weekly             Atom                           Vim  ...   
Australia       Weekly             Atom                           Vim  ...   
Austria         Weekly             Atom                           Vim  ...   
...                ...              ...                           ...  ...   
United States   Weekly             Atom                           Vim  ...   
Uruguay          Daily            Other                           Vim  ...   
Uzbekistan       Daily    IntelliJ IDEA  PyCharm Professional Edition  ...   
Venezuela      Monthly  Eclipse + Pydev                           Vim  ...   
Viet Nam        Weekly             Atom                           Vim  ...   

                              several_projects  \
                                           min   
country_live                                     
Algeria        No,  I only work on one project   
Argentina      No,  I only work on one project   
Armenia        No,  I only work on one project   
Australia      No,  I only work on one project   
Austria        No,  I only work on one project   
...                                     

In [29]:
(jb2
 .pivot_table(index='country_live',
              aggfunc={'age':['min','max'],
                       'team_size':'mean'})
)

age     team_size
              max min      mean
country_live                   
Algeria        60  18  3.722222
Argentina      60  18  4.146789
Armenia        30  18  4.235294
Australia      60  18  3.354015
Austria        50  18  3.132812
...            ..  ..       ...
United States  60  18  4.072673
Uruguay        40  21  3.700000
Uzbekistan     21  21  2.750000
Venezuela      50  18  3.227273
Viet Nam       60  18  4.666667

[76 rows x 3 columns]

In [30]:
(jb2
 .groupby('country_live')
 .agg({'age':['min','max'],
       'team_size':'mean'})
)

age     team_size
              min max      mean
country_live                   
Algeria        18  60  3.722222
Argentina      18  60  4.146789
Armenia        18  30  4.235294
Australia      18  60  3.354015
Austria        18  50  3.132812
...            ..  ..       ...
United States  18  60  4.072673
Uruguay        21  40  3.700000
Uzbekistan     21  21  2.750000
Venezuela      18  50  3.227273
Viet Nam       18  60  4.666667

[76 rows x 3 columns]

In [34]:
(jb2
 .groupby('country_live')
 .agg(age_min=('age',min),
      age_max=('age',max),
      team_size_mean=('team_size','mean')
      )
)

,age_min,age_max,team_size_mean
country_live,,,
Algeria,18,60,3.722222
Argentina,18,60,4.146789
Armenia,18,30,4.235294
Australia,18,60,3.354015
Austria,18,50,3.132812
...,...,...,...
United States,18,60,4.072673
Uruguay,21,40,3.700000
Uzbekistan,21,21,2.750000


27.5 Grouping by Hierarchy


In [39]:
(jb2
 .pivot_table(index=['country_live','ide_main'],
              values='age',
              aggfunc=[min,max])
 )

min max
                                          age age
country_live ide_main                            
Algeria      Atom                          21  60
             Eclipse + Pydev               18  18
             IDLE                          40  40
             Jupyter Notebook              30  30
             Other                         30  30
...                                        ..  ..
Viet Nam     Other                         21  21
             PyCharm Community Edition     21  30
             PyCharm Professional Edition  21  21
             VS Code                       18  30
             Vim                           21  40

[813 rows x 2 columns]

In [36]:
(jb2
 .groupby(by=['country_live','ide_main'])
 [['age']]
 .agg([min, max])
)

age      
                                                     min   max
country_live ide_main                                         
Algeria      Atom                                   21.0  60.0
             Eclipse + Pydev                        18.0  18.0
             Emacs                                   NaN   NaN
             IDLE                                   40.0  40.0
             IntelliJ IDEA                           NaN   NaN
...                                                  ...   ...
Viet Nam     Python Tools for Visual Studio (PTVS)   NaN   NaN
             Spyder                                  NaN   NaN
             Sublime Text                            NaN   NaN
             VS Code                                18.0  30.0
             Vim                                    21.0  40.0

[1216 rows x 2 columns]

In [40]:
(jb2
 .groupby(by=['country_live','ide_main'])
 [['age']]
 .agg([min, max])
 .swaplevel(axis='columns')
)

min   max
                                                     age   age
country_live ide_main                                         
Algeria      Atom                                   21.0  60.0
             Eclipse + Pydev                        18.0  18.0
             Emacs                                   NaN   NaN
             IDLE                                   40.0  40.0
             IntelliJ IDEA                           NaN   NaN
...                                                  ...   ...
Viet Nam     Python Tools for Visual Studio (PTVS)   NaN   NaN
             Spyder                                  NaN   NaN
             Sublime Text                            NaN   NaN
             VS Code                                18.0  30.0
             Vim                                    21.0  40.0

[1216 rows x 2 columns]

In [41]:
(jb2
 .groupby(by=['country_live','ide_main'])
 .agg(age_min=('age',min), age_max=('age',max))
)

age_min  age_max
country_live ide_main                                               
Algeria      Atom                                      21.0     60.0
             Eclipse + Pydev                           18.0     18.0
             Emacs                                      NaN      NaN
             IDLE                                      40.0     40.0
             IntelliJ IDEA                              NaN      NaN
...                                                     ...      ...
Viet Nam     Python Tools for Visual Studio (PTVS)      NaN      NaN
             Spyder                                     NaN      NaN
             Sublime Text                               NaN      NaN
             VS Code                                   18.0     30.0
             Vim                                       21.0     40.0

[1216 rows x 2 columns]

In [42]:
(jb2
 .groupby(by=['country_live','ide_main'], observed=True)
 .agg(age_min=('age',min), age_max=('age',max))
)

age_min  age_max
country_live       ide_main                                   
India              Atom                            18       40
                   Eclipse + Pydev                 18       40
                   Emacs                           21       40
                   IDLE                            18       40
                   IntelliJ IDEA                   21       30
...                                               ...      ...
Dominican Republic Vim                             21       21
Morocco            Jupyter Notebook                30       30
                   PyCharm Community Edition       21       40
                   Sublime Text                    21       30
                   VS Code                         21       30

[813 rows x 2 columns]

27.6 Grouping with Functions


In [43]:
def even_grouper(idx):
    return 'odd' if idx % 2 else 'even'

In [46]:
(jb2
 .pivot_table(index=even_grouper, aggfunc='size'))

even    6849
odd     6862
dtype: int64

In [44]:
(jb2
 .groupby(even_grouper)
 .size()
)

even    6849
odd     6862
dtype: int64